In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_hastie_10_2
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler 
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/ilp2021f/train_final.csv')
test = pd.read_csv('/kaggle/input/ilp2021f/test_final.csv')
test.drop(columns=['ID'], inplace=True)
test

In [ ]:
train.describe()

In [ ]:
train.isin(['?']).sum()

In [ ]:
train = train.replace('?', np.nan)
train["workclass"] = train["workclass"].fillna(train["workclass"].mode())
train["occupation"] = train["occupation"].fillna(train["occupation"].mode())
train["native.country"] = train["native.country"].fillna(train["native.country"].mode())

In [ ]:
plt.figure()

sns.displot(train['age'])
plt.ticklabel_format(style='scientific', axis='x') #repressing scientific notation on x
plt.ylabel('People')
plt.show()

In [ ]:
status = train["workclass"].value_counts(normalize=True)

sns.barplot(status.values, status.index)
plt.xlabel('People')
plt.ylabel('Workclass')
plt.tick_params(labelsize=12)
plt.show()

In [ ]:
plt.figure()

sns.displot(train['fnlwgt'])
plt.ticklabel_format(style='scientific', axis='x') #repressing scientific notation on x
plt.ylabel('People')
plt.show()

In [ ]:
status = train["education"].value_counts(normalize=True)

sns.barplot(status.values, status.index)
plt.xlabel('People')
plt.ylabel('Education')
plt.tick_params(labelsize=12)
plt.show()

In [ ]:
plt.figure()

sns.displot(train['education.num'])
plt.ticklabel_format(style='scientific', axis='x') #repressing scientific notation on x
plt.ylabel('People')
plt.show()

In [ ]:
status = train["marital.status"].value_counts(normalize=True)

sns.barplot(status.values, status.index)
plt.xlabel('People')
plt.ylabel('Marital Status')
plt.tick_params(labelsize=12)
plt.show()

In [ ]:
status = train["occupation"].value_counts(normalize=True)

sns.barplot(status.values, status.index)
plt.xlabel('People')
plt.ylabel('Occupation')
plt.tick_params(labelsize=12)
plt.show()

In [ ]:
status = train["relationship"].value_counts(normalize=True)

sns.barplot(status.values, status.index)
plt.xlabel('People')
plt.ylabel('Relationship')
plt.tick_params(labelsize=12)
plt.show()

In [ ]:
status = train["race"].value_counts(normalize=True)

sns.barplot(status.values, status.index)
plt.xlabel('People')
plt.ylabel('Race')
plt.tick_params(labelsize=12)
plt.show()

In [ ]:
status = train["sex"].value_counts(normalize=True)

sns.barplot(status.values, status.index)
plt.xlabel('People')
plt.ylabel('Sex')
plt.tick_params(labelsize=12)
plt.show()

In [ ]:
plt.figure()

sns.displot(train['capital.gain'])
plt.ticklabel_format(style='scientific', axis='x') #repressing scientific notation on x
plt.ylabel('People')
plt.show()

In [ ]:
plt.figure()

sns.displot(train['capital.loss'])
plt.ticklabel_format(style='scientific', axis='x') #repressing scientific notation on x
plt.ylabel('People')
plt.show()

In [ ]:
plt.figure()

sns.displot(train['hours.per.week'])
plt.ticklabel_format(style='scientific', axis='x') #repressing scientific notation on x
plt.ylabel('People')
plt.show()

In [ ]:
status = train["native.country"].value_counts(normalize=True)

sns.barplot(status.values, status.index)
plt.xlabel('People')
plt.ylabel('Native Country')
plt.tick_params(labelsize=8)
plt.show()

In [ ]:
for col in train.columns:
    if train[col].dtypes == 'object':
        le = LabelEncoder()
        
        train[col] = le.fit_transform(train[col].astype(str))
        
for col in test.columns:
    if test[col].dtypes == 'object':
        le = LabelEncoder()
        
        test[col] = le.fit_transform(test[col].astype(str))
#print(train.dtypes)

In [ ]:
train.corr()

In [ ]:
plt.rcParams['figure.figsize'] = [12, 12]

plt.matshow(train.corr())
plt.colorbar()
plt.xticks(np.arange(len(train.corr().columns)), train.corr().columns.values, rotation = 90) 
plt.yticks(np.arange(len(train.corr().columns)), train.corr().columns.values) 
for (i, j), corr in np.ndenumerate(train.corr()):
    plt.text(j, i, '{:0.1f}'.format(corr), ha='center', va='center', color='white', fontsize=14)
    
plt.savefig('corr.jpg')

In [ ]:
train = train[(train['capital.gain'] < 50000)]
train = train[(train['capital.loss'] < 2000)]

trainX = train.drop(["fnlwgt","occupation","native.country"],axis=1) 
test = test.drop(["fnlwgt","occupation","native.country"],axis=1) 

trainX

In [ ]:
trainX = trainX.drop(columns=['income>50K'])
trainY = train[['income>50K']]

In [ ]:
# data is very skewed
trainY.value_counts(normalize=True)

In [ ]:
rs = RandomOverSampler(random_state=20)
rs.fit(trainX,trainY)

In [ ]:
# now we have a perfect distribution
trainX,trainY = rs.fit_resample(trainX, trainY)
trainY.value_counts(normalize=True)

In [ ]:
seed = 7
test_size = 0.33
X_train, X_test, Y_train, Y_test = train_test_split(trainX, trainY, test_size=test_size, random_state=seed)

In [ ]:
xgb = XGBClassifier()
xgb = xgb.fit(X_train, Y_train.values.ravel())

predictions = xgb.predict(test)

submission = pd.DataFrame({'ID': range(1,23843), 'Prediction': predictions})
print(submission.info())
submission.to_csv('xgb4.csv', index=False)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,Y_train)

predictions = rf.predict(test)


submission = pd.DataFrame({'ID': range(1,23843), 'Prediction': predictions})
print(submission.info())
submission.to_csv('randF5.csv', index=False)